## This jupyter notebooks loads the joined pointclouds and reconstruct the items both using Poisson surface reconstruction algorithm and ball pivoting algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import trimesh

### Auxiliar functions
This block contains functions to perform the whole operation

In [ ]:
def nearest_neighbor_distance(set1, set2):
        """
        For each point in set1, find the distance to its nearest neighbor in set2.
        """
        num_points = set1.shape[0]
        nearest_distances = np.zeros(num_points)

        for i in range(num_points):
            distances = np.linalg.norm(set2 - set1[i], axis=1)
            nearest_distances[i] = np.min(distances)

        return nearest_distances
    
def compute_chamfer_distance(pc1, pc2):
    """
    Compute the Chamfer Distance between two point clouds.

    Args:
    pc1 (numpy.ndarray): First point cloud (Nx3 array).
    pc2 (numpy.ndarray): Second point cloud (Mx3 array).

    Returns:
    float: The Chamfer Distance between the two point clouds.
    """
    
    # Compute nearest neighbor distances for each direction
    distances1 = nearest_neighbor_distance(pc1, pc2)
    distances2 = nearest_neighbor_distance(pc2, pc1)

    # Compute the Chamfer Distance
    chamfer_distance = np.mean(distances1) + np.mean(distances2)

    return chamfer_distance

def bounding_boxes_intersect(mesh1, mesh2):
    """
    Check if the bounding boxes of two trimesh objects intersect.

    Args:
    mesh1, mesh2 (trimesh.Trimesh): Trimesh objects to check for intersection.

    Returns:
    bool: True if bounding boxes intersect, False otherwise.
    """
    bbox1 = mesh1.bounding_box.bounds
    bbox2 = mesh2.bounding_box.bounds

    # Check for overlap in all three dimensions
    overlap_x = bbox1[0][0] <= bbox2[1][0] and bbox1[1][0] >= bbox2[0][0]
    overlap_y = bbox1[0][1] <= bbox2[1][1] and bbox1[1][1] >= bbox2[0][1]
    overlap_z = bbox1[0][2] <= bbox2[1][2] and bbox1[1][2] >= bbox2[0][2]

    return overlap_x and overlap_y and overlap_z

def calculate_iou(mesh1_trimesh, mesh2):
    """
    Calculate the Intersection over Union (IoU) between two 3D mesh objects.

    Args:
    mesh1 (open3d.geometry.TriangleMesh): First mesh object.
    mesh2 (open3d.geometry.TriangleMesh): Second mesh object.

    Returns:
    float: The IoU value.
    """

    # Convert Open3D meshes to Trimesh objects for boolean operations
    mesh2_trimesh = trimesh.Trimesh(np.asarray(mesh2.vertices), np.asarray(mesh2.triangles))
    
    # Check if the bounding boxes of the two meshes intersect
    if not bounding_boxes_intersect(mesh1_trimesh, mesh2_trimesh):
        return 0.0

    # Compute the intersection and union of the two meshes
    intersection_mesh = mesh1_trimesh.intersection(mesh2_trimesh, engine='blender')
    union_mesh = mesh1_trimesh.union(mesh2_trimesh, engine='blender')

    # Calculate the volume of the intersection and union
    intersection_volume = intersection_mesh.volume
    union_volume = union_mesh.volume

    # Compute IoU
    iou = intersection_volume / union_volume

    return iou

def load_pcd_file(path):
    # Load the point cloud from the given file
    pcd = o3d.io.read_point_cloud(path)

    # Return the point cloud object
    return pcd

### Ball Pivoting algorithm

The ball pivoting algorithm (BPA) is a surface reconstruction method which is related to alpha shapes. Intuitively, think of a 3D ball with a given radius that we drop on the point cloud. If it hits any 3 points (and it does not fall through those 3 points) it creates triangles. Then, the algorithm starts pivoting from the edges of the existing triangles and every time it hits 3 points where the ball does not fall through we create another triangle.

**Paper:** https://ieeexplore.ieee.org/document/817351 <br>
**Open3d documentation:** https://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Ball-pivoting

In [ ]:
def reconstruct_mesh_bpa(point_cloud, radius = 5, mu=2.5, max_nn=100):
    """
    Reconstruct a 3D mesh from a point cloud using the Ball Pivoting Algorithm.

    :param point_cloud: open3d.geometry.PointCloud object, the input point cloud.
    :param radius: float, the radius of the ball used in the pivoting algorithm.
    :param mu: float, multiplier for the estimated radius (default 2.5).
    :param max_nn: int, maximum number of nearest neighbors to consider (default 100).
    :return: open3d.geometry.TriangleMesh, the reconstructed mesh.
    """

    # Estimate normals for the point cloud
    point_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius * mu, max_nn=max_nn))

    # Orient the normals (optional but recommended for better mesh quality)
    point_cloud.orient_normals_consistent_tangent_plane(k=50)

    # Reconstruct the mesh using Ball Pivoting Algorithm
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        point_cloud,
        o3d.utility.DoubleVector([radius, radius * 2, radius * 4])
    )

    return mesh

### Poisson surface reconstruction

The Poisson surface reconstruction method solves a regularized optimization problem to obtain a smooth surface. This method produces  non-smooth results since the points of the Point Cloud are also the vertices of the resulting triangle mesh without any modifications.

**Paper:** https://hhoppe.com/poissonrecon.pdf <br>
**Open3d documentation:** https://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Poisson-surface-reconstruction

In [ ]:
def reconstruct_mesh_psr(point_cloud):
    """
    Reconstruct a 3D mesh from a point cloud using the Poisson surface reconstruction algorithm.

    Args:
    point_cloud (open3d.geometry.PointCloud): The input point cloud from which the mesh will be reconstructed.

    Returns:
    open3d.geometry.TriangleMesh: The reconstructed 3D mesh.
    """

    # Estimate normals for the point cloud. This step is necessary for Poisson reconstruction.
    # The parameter 'search_param' can be adjusted depending on the density of your point cloud.
    o3d.geometry.PointCloud.estimate_normals(
        point_cloud,
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
    )

    # Compute the Poisson surface reconstruction. The 'depth' parameter controls the level of detail of the mesh.
    # Increasing 'depth' can increase the quality of the mesh but also requires more computation.
    poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        point_cloud, depth=9
    )

    # Return the reconstructed mesh
    return poisson_mesh[0]

### Main Code
This block contains the main code for the process

In [ ]:
"""
Now that we have all the pointclouds loaded into a dictionary, we can go ahead and start reconstructing the 3d models  

Dictionary that contains the predicted pointcloud and the ground truth with the following format
And another dictionary which contains the reconstructed meshes from the pointclouds
Key: Item index
Values:[Ground truth, Predicted]
"""
pointclouds_dictionary = {}
metrics_dictionary = {}
for object_index in range(1,31):
    print(f"Analyzing object {str(object_index)}")
    groundtruth_pointcloud = load_pcd_file(f'./groundtruth_pointclouds/{object_index}/merged_cloud.ply')
    groundtruth_mesh = trimesh.load(f'./groundtruth_pointclouds/{object_index}/textured.obj')
    predicted_pointcloud = load_pcd_file(f'./input_and_reflected/{object_index}/complete_pointcloud.pcd')
    
    pointclouds_dictionary[object_index] = [groundtruth_pointcloud, predicted_pointcloud]
    chamfer_distance = compute_chamfer_distance(np.array(groundtruth_pointcloud.points),np.array(predicted_pointcloud.points))
    print(chamfer_distance)
    psr_iou = calculate_iou(groundtruth_mesh, reconstruct_mesh_psr(predicted_pointcloud))
    bpa_iou = calculate_iou(groundtruth_mesh, reconstruct_mesh_bpa(predicted_pointcloud))
    metrics_dictionary[object_index] = [bpa_iou, psr_iou, chamfer_distance]

In [ ]:
###$ Code to be deleted
pointclouds_dictionary = {}
metrics_dictionary = {}
for object_index in range(1,31):
    print(f"Analyzing object {str(object_index)}")
    groundtruth_pointcloud = load_pcd_file(f'./groundtruth_pointclouds/{object_index}/merged_cloud.ply')
    groundtruth_mesh = trimesh.load(f'./groundtruth_pointclouds/{object_index}/textured.obj')
    predicted_pointcloud = load_pcd_file(f'./input_and_reflected/{object_index}/complete_pointcloud.pcd')
    
    pointclouds_dictionary[object_index] = [groundtruth_pointcloud, predicted_pointcloud]

    psr_iou = calculate_iou(groundtruth_mesh, reconstruct_mesh_psr(predicted_pointcloud))
    bpa_iou = calculate_iou(groundtruth_mesh, reconstruct_mesh_bpa(predicted_pointcloud))
    chamfer_distance = compute_chamfer_distance(np.array(groundtruth_pointcloud.points),np.array(predicted_pointcloud.points))
    metrics_dictionary[object_index] = [chamfer_distance , psr_iou, bpa_iou]